In [29]:
import torch
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

loss = nn.MarginRankingLoss(margin = 13.0)
input1 = torch.randn(3, requires_grad=True)
input2 = torch.randn(3, requires_grad=True)
target = torch.randn(3).sign()
output = loss(input1, input2, target)
print(input1, input2)
print(target)
print(output)

output.backward()


tensor([0.4860, 0.1038, 0.0934], requires_grad=True) tensor([-0.5144,  1.2280,  0.5854], requires_grad=True)
tensor([-1., -1.,  1.])
tensor(13.1227, grad_fn=<MeanBackward0>)


In [30]:
max(-target * (input1-input2)) + 13

tensor(14.0004, grad_fn=<AddBackward0>)

In [31]:

class TestDataset(Dataset):
    def __init__(self, data_nums = 10000):
        self.dataset = []
        for i in range(data_nums):
            data = {}
            data["input"] = torch.randn(1,10)
#             data["output"] = torch.tensor([[sum(data["input"][0][max(0, i-1):min(i+2, 11)]) for i in range(len(data["input"][0]))]])
            data["output"] = torch.argsort(data["input"]).float()
#             data["output"] = torch.randn(1,10) + 10
            self.dataset.append(data)
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        return self.dataset[idx]

In [32]:
class Model(nn.Module):
    def __init__(self, input_length = 10, output_length = 10):
        super(Model, self).__init__()

        self.seq = nn.Sequential(
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
        )
#         self.Conv2d = nn.Conv2d(1,1,kernel_size=3,padding=1)
    def forward(self, data):
        out = self.seq(data)
        return out
        

In [42]:
3**27

7625597484987

In [33]:
dataset = TestDataset()
model = Model()

In [34]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(model.parameters(), lr=0.0001)
MSE = nn.MSELoss()
CS = nn.CosineSimilarity(dim = 2)

data_loader = torch.utils.data.DataLoader(dataset, batch_size = 4096, shuffle = True)
model.to(device)


Model(
  (seq): Sequential(
    (0): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (2): Tanh()
    (3): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): Tanh()
    (6): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (7): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (8): Tanh()
    (9): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (10): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (11): Tanh()
    (12): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (13): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
  )
)

In [37]:
from tqdm import tqdm
import numpy as np
import copy

before_param_data = {}


for name, param in model.named_parameters():
    if param.requires_grad:
        before_param_data[name] = copy.deepcopy(param.data)
        
for epoch in range(1000000):
    loss_mean = []
    for i, batch in (enumerate(data_loader)):
        input_data, GT = (batch["input"].to(device), batch["output"].to(device))

        optimizer.zero_grad()
        predictTE = model(input_data)
        sort_idx = torch.argsort(predictTE, dim = 2).float()
#         predictTE = predictTE * 0 + sort_idx


        loss = MSE(sort_idx, GT)
#         loss = loss + torch.mean(1. - (CS(predictTE, GT))) * 5
#         print(input_data.shape)
#         print(GT.shape)
#         assert False, "AA"
#         print(GT[0])
#         print(loss.shape)
#         print(torch.sum(loss))
        loss.backward()
        optimizer.step()
        
        loss_mean.append(loss.cpu().item())
    if epoch % 20 == 19:
        print(np.mean(loss_mean))
        for name, param in model.named_parameters():
            if param.requires_grad:
#                 print(name, param.data)
#                 print(name, before_param_data[name])
#                 print(predictTE[0])
#                 print(GT[0])

#                 print((before_param_data[name] == param.data).any())
#                 assert (before_param_data[name] != param.data).any(), "BACK PROPAGATION FAIL!!!!!!!!!!!!!!!!!"
#                 break
                pass
        print(predictTE[0])
        print(torch.argsort(predictTE[0]).float())
        print(GT[0])
#         print(loss.requires_grad)
        print("=" * 80)

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
model.requires_grad

In [ ]:
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)
output.shape

In [ ]:
(before_param_data[name] == param.data).any()

In [ ]:
param.data

In [ ]:
assert False, "AA"

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)